In [1]:
print("Hello")

Hello


In [2]:
# %%capture
# !pip install unsloth
# # Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [3]:
!pip uninstall torch torchvision torchaudio -y  # Remove existing PyTorch
!pip cache purge
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118  # Install PyTorch with CUDA 11.8
!pip install -U unsloth
!pip install -U accelerate
!pip install -U bitsandbytes
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Found existing installation: torchvision 0.19.0
Uninstalling torchvision-0.19.0:
  Successfully uninstalled torchvision-0.19.0
Found existing installation: torchaudio 2.4.0
Uninstalling torchaudio-2.4.0:
  Successfully uninstalled torchaudio-2.4.0
Files removed: 8
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.3/838.3 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 57.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 79.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### Inference
We use the model. We use TextStreamer for continuous inference, token by token.

temperature=1.5: Controls randomness in the generation. Higher values like 1.5 produce more diverse and creative outputs.
min_p=0.1: Filters out tokens with a cumulative probability lower than 0.1, ensuring less likely tokens are excluded for more coherent results.

In [5]:
from unsloth import FastLanguageModel
model_load, tokenizer_load = FastLanguageModel.from_pretrained(
    model_name = "/kaggle/input/chatcbb1epoch/transformers/default/1/lora_model_1_epoch", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2024.10.3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu118. CUDA = 7.5. CUDA Toolkit = 11.8.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`
Unsloth 2024.10.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [37]:
from IPython.display import Markdown, display

FastLanguageModel.for_inference(model_load) # Enable native 2x faster inference

prompt = "hello?"

messages = [
    {"role": "user", "content": prompt},
]
inputs = tokenizer_load.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt", #returns as tensor
).to("cuda") #Uses GPU for inference

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer_load, skip_prompt = True)
output = model_load.generate(input_ids = inputs, streamer = None, max_new_tokens = 256, #Limits the generated text to 128 new tokens.
                   use_cache = True, temperature = 0.5, min_p = 0)

# Decode the output tokens to get the generated string
decoded_output = tokenizer_load.decode(output[0], skip_special_tokens=True)

# Optionally, remove any unwanted system messages
cleaned_output = decoded_output.split("2024", 1)[-1].strip()

# Ensure the assistant's response starts on a new line
final_output = cleaned_output.replace("assistant", "\n\nassistant")  # Add newline before 'assistant'

# Display the cleaned output in Markdown format
display(Markdown(final_output))

user

hello?

assistant

It looks like you are asking about the meaning of "data mining".  I am going to answer your question by saying that "data mining" is a term used to describe the process of extracting useful information from data.  It is a process that is used in many fields of science and engineering, and is a term that is often used to describe the process of extracting useful information from data in the field of statistics.  It is also a term that is often used in the field of artificial intelligence.
In the field of statistics, data mining is often used to describe the process of extracting useful information from data.  It is a process that is used to extract useful information from data, and is often used to identify patterns and trends in data.  It is also a process that is used to identify useful information in data, and is often used to identify patterns and trends in data.  In the field of artificial intelligence, data mining is often used to describe the process of extracting useful information from data, and is often used to identify patterns and trends in data.  It is a process that is used to extract useful information from data, and is often used to identify patterns and trends in data.
In addition, data mining is often used to describe the process of extracting useful